# Light FM 近似最近傍探索 Approximate Nearest Neighbours (ANN)

このノートブックでは、例の movielens データセット上にハイブリッド LightFM モデルを構築します。[LightFM](https://github.com/lyst/lightfm)パッケージを使用して純粋な行列分解とハイブリッドモデルを構築する手順を概説しています。また、適合したハイブリッドモデルからユーザーとアイテムの類似性の両方を抽出する方法も示しています。より詳細な LightFM モデルの構築は、[Microsoft Recommenders](https://github.com/microsoft/recommenders/blob/master/examples/02_model_hybrid/lightfm_deep_dive.ipynb) を参照してください。

次に、アイテムとユーザのベクトルを格納し、スケールでの高速なクエリのために、これらを対象とした ANN インデックスを構築します。この手法では、何百万ものアイテムに対して 50 ミリ秒以下の応答時間を実現できます。

2 つの人気のある ANN ライブラリ、Annoy と NMSlib を見て、その性能を比較します。

依存関係:
- Annoy - https://github.com/spotify/annoy
- NMSLIB - https://github.com/searchivarius/nmslib/tree/master/python_bindings

まずはデータをインポートして、ドキュメントの例のように Light FM モデルを構築してみましょう。

In [ ]:
# lightfm ライブラリのインストール
! pip install lightfm

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances
import time
from lightfm.datasets import fetch_movielens
import warnings
warnings.simplefilter('ignore')

movielens = fetch_movielens()

In [ ]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

In [ ]:
train = movielens['train']
test = movielens['test']

In [ ]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)

model.fit_partial(train, item_features=movielens['item_features'], epochs=20 )

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

項目の特徴量にモデルの項目特徴量を乗算するだけで、項目のエンベッディングを得ることができます。さらに良いことに、LightFM には、特徴量の集合を与えて、これらのエンベッディングを取得するための機能が組み込まれています。

In [ ]:
_, item_embeddings = model.get_item_representations(movielens['item_features'])

それでは、項目間の問い合わせのための Annoy インデックスを作ってみましょう。

In [ ]:
from annoy import AnnoyIndex

factors = item_embeddings.shape[1] #インデックス化される項目ベクトルの長さ
annoy_idx = AnnoyIndex(factors)  
for i in range(item_embeddings.shape[0]):
    v = item_embeddings[i]
    annoy_idx.add_item(i, v)

annoy_idx.build(10) # 10 trees
annoy_idx.save('movielens_item_Annoy_idx.ann')

そして、類似したムービーのインデックスを検索します。

In [ ]:
def nearest_movies_annoy(movie_id, index, n=10, print_output=True):
    nn = index.get_nns_by_item(movie_id, 10)
    if print_output:
        print('Closest to %s : \n' % movielens['item_labels'][movie_id])
    titles = [movielens['item_labels'][i] for i in nn]
    if print_output:
        print("\n".join(titles))

In [ ]:
nearest_movies_annoy(90, annoy_idx)

悪くはない結果です。そして高速な処理速度が得られます。

In [ ]:
%%timeit
nearest_movies_annoy(90, annoy_idx, print_output=False)

NMSlib はどうでしょうか？

In [ ]:
import nmslib

# コサイン類似度のHNSWインデックスを用いて、新しいnmslibインデックスを初期化します。
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [ ]:
def nearest_movies_nms(movie_id, index, n=10, print_output=True):
    nn = index.knnQuery(item_embeddings[movie_id], k=10)
    if print_output == True:
        print('Closest to %s : \n' % movielens['item_labels'][movie_id])
    titles = [movielens['item_labels'][i] for i in nn[0]]
    if print_output == True:
        print("\n".join(titles))

In [ ]:
nearest_movies_nms(90, nms_idx, n=10)

In [ ]:
%%timeit 
nearest_movies_nms(90, nms_idx, n=10, print_output=False)

またしても、良い、おそらく少し良くなっている、また非常に高速です。さて、Xbox のレコメンデーション チームによって概説された巧妙なトリックを使用して、ユーザーのレコメンデーションを行う方法を示す例を紹介します: https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/XboxInnerProduct.pdf. Ben Fredrickson https://github.com/benfred/ に感謝します。

基本的には、各項目ベクトルに正規化係数を追加して、その距離を互いに等しくします。そして、ユーザ・ベクトルで問い合わせを行うとき、最後に 0 を追加し、結果はユーザ・ベクトルとアイテム・ベクトルの内積に比例します。これは，近似最大内積検索を行う COOL な方法です．

In [ ]:
norms = np.linalg.norm(item_embeddings, axis=1)
max_norm = norms.max()
extra_dimension = np.sqrt(max_norm ** 2 - norms ** 2)
norm_data = np.append(item_embeddings, extra_dimension.reshape(norms.shape[0], 1), axis=1)

#First an Annoy index:

user_factors = norm_data.shape[1]
annoy_member_idx = AnnoyIndex(user_factors)  # Length of item vector that will be indexed

for i in range(norm_data.shape[0]):
    v = norm_data[i]
    annoy_member_idx.add_item(i, v)
    
annoy_member_idx.build(10)

# Now an NMS index

nms_member_idx = nmslib.init(method='hnsw', space='cosinesimil')
nms_member_idx.addDataPointBatch(norm_data)
nms_member_idx.createIndex(print_progress=True)

In [ ]:
# ユーザーベクトルを定義する

_, user_embeddings = model.get_user_representations()

#### これで、標準のLightFMの例のようにクエリを実行することができます。

In [ ]:
def sample_recommendation(user_ids, model, data, n_items=10, print_output=True):
    n_users, n_items = data['train'].shape

    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        top_items = [data['item_labels'][i] for i in annoy_member_idx.get_nns_by_vector(np.append(user_embeddings[user_id], 0), 50)]
        if print_output == True:
            print("User %s" % user_id)
            print("     Known positives:")

            for x in known_positives[:3]:
                print("        %s" % x)

            print("     Recommended:")

            for x in top_items[:3]:
                print("        %s" % x)

In [ ]:
sample_recommendation([3,25,450], model, movielens, print_output=True)

#### ここでオリジナルの非 ANN 版と比較してみてください。

In [ ]:
def sample_recommendation_original(model, data, user_ids, print_output=True):

    n_users, n_items = data['train'].shape

    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels'][np.argsort(-scores)]
        if print_output == True:
            print("User %s" % user_id)
            print("     Known positives:")

            for x in known_positives[:3]:
                print("        %s" % x)

            print("     Recommended:")

            for x in top_items[:3]:
                print("        %s" % x)

In [ ]:
sample_recommendation_original(model, movielens, [3, 25, 450])

かなり似ていますね！

## どのくらいの速さでスケールするのでしょうか？

ここでは、1 つの予測で比較した 2 つの機能を紹介します。

In [ ]:
%%timeit
sample_recommendation_original(model, movielens, [3, 25, 450], print_output=False)

In [ ]:
%%timeit
sample_recommendation([3,25,450], model, movielens, print_output=False)

そのため、これは少し速くなりますが、これらのボリュームでは、おそらく余分な努力を正当化するものではないでしょう。これらのライブラリが価値を発揮するのは、非常に大量のベクトルに対してです。では、ANN ライブラリと Scikit-Learn のペアワイズ距離関数との比較を見てみましょう。

In [ ]:
def time_nearest_neighbours_methods(item_embeddings):
    
    # Pairwise distances nearest neighbours
    start = time.perf_counter()
    distances = pairwise_distances(item_embeddings[0].reshape(1, -1), Y=item_embeddings)
    top_items = distances.argsort()[:10]
    end = time.perf_counter()
    pairwise_elapsed = end - start
    
    #NMS lib
    nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
    nms_idx.addDataPointBatch(item_embeddings)
    nms_idx.createIndex(print_progress=False)
    start = time.perf_counter()
    nms_idx.knnQuery(item_embeddings[0])[0]
    end =time.perf_counter()
    nms_elapsed = end - start

    #Annoy
    annoy_idx = AnnoyIndex(item_embeddings.shape[1])
    for i in range(item_embeddings.shape[0]):
        v = item_embeddings[i]
        annoy_idx.add_item(i, v)
    annoy_idx.build(10) # 10 trees
    
    start = time.perf_counter()
    annoy_idx.get_nns_by_vector(item_embeddings[0], 10)
    end = time.perf_counter()    
    annoy_elapsed =  end - start
   
    return {'pairwise_elapsed': pairwise_elapsed, 'nms_elapsed': nms_elapsed, 'annoy_elapsed' : annoy_elapsed}

pairwise_timings = []
nms_timings = []
annoy_timings = []

for i in range (1, 100, 10):
    loop_item_embeddings = np.repeat(item_embeddings, i, axis=0)
    results = time_nearest_neighbours_methods(loop_item_embeddings)
    pairwise_timings.append(results['pairwise_elapsed'])
    nms_timings.append(results['nms_elapsed'])
    annoy_timings.append(results['annoy_elapsed'])
    
timings_df = pd.DataFrame({'Pairwise_dist':pairwise_timings, \
                           'NMS': nms_timings, \
                           'Annoy': annoy_timings}, \
                          index=np.arange(10)*item_embeddings.shape[0])

import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=1, ncols=2)
ax = timings_df.plot(figsize=(14,4), title='Time to compute nearest neighbours vs. number of vectors', ax=axes[0])
ax.set(xlabel='Numer of vectors', ylabel='Time (ms)')

ax2 = timings_df.loc[:, 'NMS':'Annoy'].plot(figsize=(14,4), title='Time to compute nearest neighbours vs. number of vectors', ax=axes[1])
ax2.set(xlabel='Numer of vectors', ylabel='Time (ms)')

ご覧のように、NMSLib と Annoy の両方とも、より多くのベクタ数でペアワイズ距離法よりもはるかに高速です。実際、これらのボリュームでは、ベクタ数を増やしても応答時間の増加は全く見られません。

## では、ANNライブラリの使用は品質にどのように影響するのでしょうか？

アイデアを得るために、Annoy と NMS インデックスを使って予測値を計算し、LightFM のビルトイン予測メソッドと比較して、それらの precision@k を評価してみましょう。

In [ ]:
lightfm_pak = precision_at_k(model, test, k=10).mean()

def annoy_precision_at_k(user_ids, k=10):
    test_csr = test.tocsr()
    paks = []
    for user_id in user_ids:
        test_interactions = test_csr[user_id].indices
        if len(test_interactions) > 0:
            recommendations = annoy_member_idx.get_nns_by_vector(np.append(user_embeddings[user_id], 0), k)
            hits = len(set(test_interactions).intersection(recommendations))
            pak = hits / (len(test_interactions) * 1.0)
            paks.append(pak)
    return np.array(paks).mean()

annoy_pak = annoy_precision_at_k(np.arange(test.shape[0]))

def nms_precision_at_k(user_ids, k=10):
    test_csr = test.tocsr()
    paks = []
    for user_id in user_ids:
        test_interactions = test_csr[user_id].indices
        if len(test_interactions) > 0:
            recommendations = nms_member_idx.knnQuery(np.append(user_embeddings[user_id], 0), k=10)[0]
            hits = len(set(test_interactions).intersection(recommendations))
            pak = hits / (len(test_interactions) * 1.0)
            paks.append(pak)
    return np.array(paks).mean()

nms_pak = nms_precision_at_k(np.arange(test.shape[0]))

plt.bar(['LightFm', 'Annoy', 'NMSLib'], [lightfm_pak, annoy_pak, nms_pak])
plt.title('Precision at K=10, ANN vs. exact')
plt.ylabel('Precision at k=10')
plt.show()

## では、これらのライブラリのどれを使うべきでしょうか？

小規模なデータセットでは、わずかな速度向上のために手間をかけたり精度を落としたりする価値はありません。しかし、100ms 以下で応答する必要がある場合は、これらのライブラリのいずれかを検討する必要がある前に、ベクトルセットが大きくなるだけです。

Annoy はメモリマップに保存できるという利点があるので、本番環境では複数のプロセスでも問題なく動作します。

NMSlib は精度の面では Annoy を凌駕していますが、大規模なデータセットのインデックス構築には時間がかかることもわかりました。これは、動きの速い製品カタログなどのために頻繁にインデックスを再構築しなければならない場合に問題になります。